In [5]:
import nbimporter
import numpy as np
from WaveNetClassifier import WaveNet, WaveNetParams

Importing Jupyter notebook from WaveNetClassifier.ipynb


Using TensorFlow backend.


Importing Jupyter notebook from DataParser.ipynb


In [4]:
def evaluateTestSet(net, weight_path):
    if net == 'wavent':
        model = WaveNet()
    model = model.load_weights(weight_path)
    result = model.evaluate(x_test, y_test, batch_size=2048)
    print('Accuracy: %2.1f%%, Top 3 Accuracy %2.1f%%' % (100*lstm_results[1], 100*lstm_results[2]))

In [1]:
def sklearnReport(model, weight_path):
    model = model.load_weights(weight_path)
    test_cat = np.argmax(y_test, 1)
    pred_y = model.predict(x_test, batch_size = 4096)
    pred_cat = np.argmax(pred_y, 1)
    plt.matshow(confusion_matrix(test_cat, pred_cat))
    print(classification_report(test_cat, pred_cat, 
                            target_names = [x for x in word_encoder.classes_]))